In [1]:
#!pip install citipy

In [2]:
import matplotlib.pyplot as plt
import requests
import pandas as pd
import numpy as np
import time
from scipy.stats import linregress
# Import api key
from config import api_key
# Import citipy for city names by long. and lat.
from citipy import citipy

In [3]:
# Define ranges for longitude and latitude
latitude_range = (-90,90)
longitude_range = (-180,180)

In [4]:
# Create lists for coordinates and cities
lat_long_coord = []
cities = []

# Create random latitude and longitude combinations
latitude = np.random.uniform(latitude_range[0], latitude_range[1], size=1500)
longitude = np.random.uniform(longitude_range[0], longitude_range[1], size=1500)
lat_long_coord = zip(latitude, longitude)

# Build a for loop to find city names for random coordinates
for lat_long in lat_long_coord:
    city = citipy.nearest_city(lat_long[0], lat_long[1])
    city = city.city_name
    
    # Add the city to a list if it is unique
    if city not in cities:
        cities.append(city)

In [5]:
# Check length of the list
len(cities)
#print(cities)

606

In [6]:
# Create base url
base_url = f"http://api.openweathermap.org/data/2.5/weather?appid={api_key}"
# define units
units = "imperial"

# Build query url with units
query_url = f"{base_url}&units={units}&q="

In [7]:
# Create lists for results
temperature = []
humidity = []
cloudiness = []
wind_speed = []
countries = []
date = []
cities_reduced = []


In [8]:
# Loop through cities to find results
index = 0

for city in cities:
    
    # Create url for city
    full_url = f"{query_url}{city}"
    index += 1
    
    try:
    # Find json response for city
        response = requests.get(full_url).json()
        city_number = response["id"]
        
        # Find data to append for each city
        temp_max = response["main"]["temp_max"]
        humid = response["main"]["humidity"]
        clouds = response["clouds"]["all"]
        wind = response["wind"]["speed"]
        country = response["sys"]["country"]
        weather_date = ["dt"]
        
         # Append results
        print(f"City {index} of {len(cities)}: {city} found. City Number: {city_number}. Appending results.")
        
        temperature.append(temp_max)
        humidity.append(humid)
        cloudiness.append(clouds)
        wind_speed.append(wind)
        cities_reduced.append(city)
        countries.append(country)
        date.append(date)
        
    except KeyError:
        print(f"{city} not found.")
        

City 1 of 606: arraial do cabo found. City Number: 3471451. Appending results.
City 2 of 606: puerto ayora found. City Number: 3652764. Appending results.
City 3 of 606: mataura found. City Number: 6201424. Appending results.
City 4 of 606: albany found. City Number: 5106841. Appending results.
City 5 of 606: gornopravdinsk found. City Number: 1506268. Appending results.
City 6 of 606: hilo found. City Number: 5855927. Appending results.
City 7 of 606: pisco found. City Number: 3932145. Appending results.
City 8 of 606: busselton found. City Number: 2075265. Appending results.
City 9 of 606: castro found. City Number: 3466704. Appending results.
City 10 of 606: ushuaia found. City Number: 3833367. Appending results.
City 11 of 606: marquette found. City Number: 5000950. Appending results.
attawapiskat not found.
City 13 of 606: kundiawa found. City Number: 2093059. Appending results.
City 14 of 606: fortuna found. City Number: 5563839. Appending results.
belushya guba not found.
City 1

City 119 of 606: laerdalsoyri found. City Number: 3148569. Appending results.
City 120 of 606: nakamura found. City Number: 1855891. Appending results.
City 121 of 606: butaritari found. City Number: 2110227. Appending results.
City 122 of 606: kolpashevo found. City Number: 1502862. Appending results.
City 123 of 606: hithadhoo found. City Number: 1282256. Appending results.
illoqqortoormiut not found.
City 125 of 606: agadez found. City Number: 2448083. Appending results.
City 126 of 606: longyearbyen found. City Number: 2729907. Appending results.
City 127 of 606: miramar found. City Number: 4164601. Appending results.
City 128 of 606: chiredzi found. City Number: 893485. Appending results.
City 129 of 606: mayumba found. City Number: 2399001. Appending results.
City 130 of 606: lasa found. City Number: 3175096. Appending results.
City 131 of 606: awbari found. City Number: 2219235. Appending results.
City 132 of 606: maicao found. City Number: 3675657. Appending results.
City 133 o

City 239 of 606: gunjur found. City Number: 2413419. Appending results.
City 240 of 606: hryshkivtsi found. City Number: 708057. Appending results.
yunjinghong not found.
City 242 of 606: sechura found. City Number: 3691954. Appending results.
City 243 of 606: muswellbrook found. City Number: 2156034. Appending results.
City 244 of 606: dunedin found. City Number: 2191562. Appending results.
City 245 of 606: east london found. City Number: 1006984. Appending results.
City 246 of 606: naraina found. City Number: 1261872. Appending results.
City 247 of 606: khatanga found. City Number: 2022572. Appending results.
City 248 of 606: ampanihy found. City Number: 1078553. Appending results.
City 249 of 606: olinda found. City Number: 3393536. Appending results.
City 250 of 606: tigil found. City Number: 2120612. Appending results.
City 251 of 606: meulaboh found. City Number: 1214488. Appending results.
City 252 of 606: sao filipe found. City Number: 3374210. Appending results.
City 253 of 60

City 358 of 606: key largo found. City Number: 4160795. Appending results.
City 359 of 606: tuatapere found. City Number: 2180815. Appending results.
City 360 of 606: kainantu found. City Number: 2095038. Appending results.
City 361 of 606: buguruslan found. City Number: 571159. Appending results.
City 362 of 606: umea found. City Number: 602150. Appending results.
City 363 of 606: makakilo city found. City Number: 5850554. Appending results.
mys shmidta not found.
City 365 of 606: grevesmuhlen found. City Number: 2917520. Appending results.
City 366 of 606: maragogi found. City Number: 3395458. Appending results.
City 367 of 606: caibarien found. City Number: 3566356. Appending results.
City 368 of 606: huinan found. City Number: 2036685. Appending results.
City 369 of 606: yeppoon found. City Number: 2142316. Appending results.
City 370 of 606: the valley found. City Number: 3573374. Appending results.
City 371 of 606: penzance found. City Number: 2640377. Appending results.
City 372

City 475 of 606: morant bay found. City Number: 3489440. Appending results.
City 476 of 606: katangli found. City Number: 2122783. Appending results.
City 477 of 606: walvis bay found. City Number: 3359638. Appending results.
City 478 of 606: urumqi found. City Number: 1529102. Appending results.
City 479 of 606: buraydah found. City Number: 107304. Appending results.
City 480 of 606: gilford found. City Number: 5086544. Appending results.
City 481 of 606: wewak found. City Number: 2083537. Appending results.
City 482 of 606: poum found. City Number: 2138555. Appending results.
City 483 of 606: aswan found. City Number: 359792. Appending results.
City 484 of 606: vysokogornyy found. City Number: 2013216. Appending results.
City 485 of 606: skibbereen found. City Number: 2961459. Appending results.
marcona not found.
City 487 of 606: sampit found. City Number: 1628884. Appending results.
City 488 of 606: vrangel found. City Number: 2013258. Appending results.
City 489 of 606: quartucciu

City 593 of 606: naron found. City Number: 3115739. Appending results.
City 594 of 606: pandan found. City Number: 1695546. Appending results.
City 595 of 606: igualada found. City Number: 3120431. Appending results.
City 596 of 606: aksarka found. City Number: 1512019. Appending results.
City 597 of 606: synya found. City Number: 485215. Appending results.
City 598 of 606: bulalacao found. City Number: 1723002. Appending results.
tumannyy not found.
City 600 of 606: zhezkazgan found. City Number: 1516589. Appending results.
City 601 of 606: hualmay found. City Number: 3939761. Appending results.
City 602 of 606: goianesia found. City Number: 3462378. Appending results.
City 603 of 606: kuching found. City Number: 1735634. Appending results.
City 604 of 606: bani found. City Number: 3512067. Appending results.
City 605 of 606: zeya found. City Number: 2012593. Appending results.
City 606 of 606: rosarno found. City Number: 2523585. Appending results.


In [16]:
# Create DataFrame from series
weather_data = {
    "City": cities_reduced,
    "Latitude": latitude,
    "Longitude": longitude,
    "Max Temp.": temperature,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country": country,
    "Dates": weather_date
}
#weather_data = pd.DataFrame(weather_data)

In [ ]:
print(len(cities_reduced))
print(len(latitude))
print(len(longitude))
print(len(temperature))
print(len(humidity))
print(len(cloudiness))
print(len(wind_speed))
print(len)


In [ ]:
# Output dataframe to csv
weather_data.to_csv("./Output_Data/cities_weather_data.csv", encoding="utf-8")

In [ ]:
# Bring dataframe back in
weather_data_df = pd.read_csv("./Output_Data/cities_weather_data.csv")